<a href="https://colab.research.google.com/github/dkdevAI/imageClassification/blob/main/Residual%20Neural%20Network/customResnet11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, GlobalAveragePooling2D, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K

# Load the Fashion MNIST dataset
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Preprocess the data
train_images = train_images.reshape(-1, 28, 28, 1).astype('float32') / 255.0
test_images = test_images.reshape(-1, 28, 28, 1).astype('float32') / 255.0

train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

# Define the residual block
def residual_block(input_tensor, filters, strides=(1, 1), use_shortcut=False):
    x = Conv2D(filters, (3, 3), strides=strides, padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)

    if use_shortcut:
        shortcut = Conv2D(filters, (1, 1), strides=strides, padding='same')(input_tensor)
        shortcut = BatchNormalization()(shortcut)
        x = Add()([x, shortcut])
    else:
        x = Add()([x, input_tensor])

    x = Activation('relu')(x)
    return x

# Build the ResNet model
def build_resnet11():
    input_shape = (28, 28, 1)
    inputs = Input(shape=input_shape)

    x = Conv2D(64, (3, 3), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Stack of residual blocks
    x = residual_block(x, 64, use_shortcut=True)
    for _ in range(2):
        x = residual_block(x, 64)

    x = residual_block(x, 128, strides=(2, 2), use_shortcut=True)
    for _ in range(2):
        x = residual_block(x, 128)

    x = GlobalAveragePooling2D()(x)
    outputs = Dense(10, activation='softmax')(x)

    model = Model(inputs, outputs)
    return model

# Create and compile the model
model = build_resnet11()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model
model.fit(train_images, train_labels, batch_size=128, epochs=5, validation_split=0.1)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print("Test Accuracy:", test_accuracy)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 28, 28, 64)   640         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 28, 28, 64)  256         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 28, 28, 64)   0           ['batch_normalization[0][0]']